<a href="https://colab.research.google.com/github/huricane85/fpl2020_python/blob/master/FPL_API_amosbastian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/177arc/pandas-fpl

Cloning into 'pandas-fpl'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 144 (delta 59), reused 118 (delta 40), pack-reused 0
Receiving objects: 100% (144/144), 49.89 KiB | 7.13 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
!git clone https://github.com/drew7vtedu/fpl_player_analysis

Cloning into 'fpl_player_analysis'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 4), reused 11 (delta 4), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [4]:
import csv
import player_object
import fantasy_value_metrics as metrics

def save_db(players):

    csv_columns = ["first_name", "last_name", "points_last_season", "current_points", "price", "minutes", "points_per_90", "points_per_match", "points_per_match2", "value_added_per_mil"]

    outfile = open("./Data/saved_player_db.csv", 'w')
    # write headers for each column of the csv
    writer = csv.DictWriter(outfile, fieldnames = csv_columns)
    writer.writeheader()
    for player in players:
        writer.writerow(player.to_dict())



# create the initial player db and save it to a csv which can be easily loaded
def create_player_db(last_year_csv, this_year_csv):
    # open both files with the provided paths
    last_year = open(last_year_csv)
    this_year = open(this_year_csv)

    last_parser = csv.reader(last_year)
    current_parser = csv.reader(this_year)

    # skip headers
    next(last_parser)
    next(current_parser)

    # list of player objects
    player_db = []


    # first fill db with this years players
    for row in current_parser:
       player = player_object.player_obj(row[0], row[1], 0, 0, int(row[3]), 0 row[5])
       player_db.append(player)


    # check each player in last years data against current list and update matches
    for row in last_parser:
        for player in player_db:
            if player.matches(row[0], row[1]):
                player.set_pls(int(row[4]))
                player.set_minutes(int(row[5]))
                player.set_points_per_90(metrics.points_scored_per_90(int(row[4]), int(row[5])))
                player.set_points_per_match(metrics.points_scored_per_match(int(row[4]), 38))
                player.set_points_per_match2(metrics.points_scored_per_match(int(row[4]) - (2 * 38), 38))
                player.set_value_added_per_mil(metrics.value_added_per_mil(int(row[4]), 38, player.get_price()))

    last_year.close()
    this_year.close()
    save_db(player_db)

# driver code
create_player_db("./Data/cleaned_players_19-20.csv", "./Data/current_player_data.csv")


SyntaxError: ignored

In [10]:
import pandas as pd
import json
import csv
import requests
from pandas.io.json import json_normalize




# Define a function to get info from the FPL API and save to the specified file_path
# It might be a good idea to navigate to the link in a browser to get an idea of what the data looks like
def get_json(file_path):
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    jsonResponse = r.json()
    with open(file_path, 'w') as outfile:
        json.dump(jsonResponse, outfile)

# Run the function and choose where to save the json file
get_json('player_data.json')

# open the json file containing your data
json_data = open('player_data.json', encoding='utf-8')

# get a string (text) representation of the entire contents of the file
data = json_data.read()

# convert text to a python dictionary of key-value pairs
data_dict = json.JSONDecoder().decode(data)

# done with json input file
json_data.close()

players = data_dict["elements"]

# we will fill this list with data and convert it to csv later
csv_player_list = []
# column headers for when we convert to csv
csv_columns = ["first_name", "last_name", "current_points", "price", "minutes", "team", "Points per game"]

for player in players:
    # create dict to fill with each player's data
    temp_player_dict = {}
    # fill temp dict with desired data
    temp_player_dict["first_name"] = player["first_name"]
    temp_player_dict["last_name"] = player["second_name"]
    temp_player_dict["current_points"] = int(player["total_points"])
    temp_player_dict["price"] = int(player["now_cost"])
    temp_player_dict["minutes"] = int(player["minutes"])
    temp_player_dict["Points per game"] = (player["points_per_game"])

    # add data to list to become csv
    csv_player_list.append(temp_player_dict)

# name of the csv file to write to
csv_out = "current_player_data.csv"
try:
    with open(csv_out, 'w') as csvfile:
        # write headers for each column of the csv
        writer = csv.DictWriter(csvfile, fieldnames = csv_columns)
        writer.writeheader()
        for player in csv_player_list:
            writer.writerow(player)
except IOError:
    print("IOError while opening csv file")
# close out file
csvfile.close()

# data I can get from each player:
# 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type'


https://gist.github.com/calumrussell/9a869765af0698fd4fa934ca90029fc8